In [53]:
# LLM tentative

In [54]:
!pip install tensorflow

In [55]:
!pip install transformers

In [56]:
!pip install torch torchvision torchaudio

In [57]:
# import section

import yfinance as yf
import numpy as np
import torch
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TFGPT2LMHeadModel

In [58]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.7898, 0.9316, 0.1986],
        [0.4891, 0.5549, 0.1309],
        [0.3374, 0.7846, 0.9932],
        [0.5682, 0.7090, 0.6061],
        [0.7336, 0.5782, 0.3954]])


In [59]:
import torch
torch.cuda.is_available()

False

In [60]:
ticker = "^NSEI"
start_date = "2023-01-01"
end_date = "2023-06-08"
data = yf.download(ticker, start=start_date, end=end_date)

prices = data["Close"].tolist()

[*********************100%%**********************]  1 of 1 completed


In [61]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [62]:
# model = GPT2LMHeadModel.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [63]:
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
encoded_prices = tokenizer.encode(" ".join([str(price) for price in prices]), return_tensors="pt")




ImportError: Unable to convert output to PyTorch tensors format, PyTorch is not installed.

In [ ]:
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.resize_token_embeddings(len(tokenizer))

for _ in range(3):
    model.zero_grad()
    outputs = model(encoded_prices, labels=encoded_prices)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

In [ ]:
generated = model.generate(encoded_prices, max_length=len(encoded_prices) + 10, temperature=1.0, num_return_sequences=1)
generated_prices = tokenizer.decode(generated[0], skip_special_tokens=True).split()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(data.index, prices, label="Historical Prices")
plt.plot(data.index[-1] + pd.to_timedelta(np.arange(1, len(generated_prices) + 1), 'D'), [float(price) for price in generated_prices[len(prices):]], "g^", label="Predicted Prices")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.title(f"{ticker} - Historical and Predicted Stock Prices (GPT)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()